In [3]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow

In [4]:
train = pd.read_csv('../data/train.csv')

In [5]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [6]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Hyper-param optmization

In [7]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow
import warnings

warnings.filterwarnings('ignore', message='.*Falling back to prediction using DMatrix.*')

def objective(trial):
    with mlflow.start_run(run_name=f'Trial - {trial.number}', nested=True):
        # Define aqui os hyperparâmetros do RandomForest
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
            'max_depth': trial.suggest_int('max_depth', 3, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5, 0.75]),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
            'random_state': 42,
            'n_jobs': 1
        }

        mlflow.log_params(params)

        # Instancia o modelo RF
        model = RandomForestRegressor(**params)

        # Usa RMSE negativo como métrica para o cross_val_score
        scoring = 'neg_root_mean_squared_error'
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        scores = cross_val_score(
            model, X, y,
            scoring=scoring,
            cv=kf
        )

        rmse = -scores.mean()  # torna positivo
        mlflow.log_metric("rmse", rmse)

        return rmse


c:\Users\Guilherme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=20,
        n_warmup_steps=10,
        interval_steps=5
    )
)

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('DSWA - Competicao3')
with mlflow.start_run(run_name='Random Forest optimization'):
    study.optimize(objective, n_trials=50)
    
    print("Melhores hiperparâmetros:")
    print(study.best_params)
    print(f"Melhor RMSE: {study.best_value:.4f}")

    # Loga os melhores parâmetros e métrica no MLflow
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)


[I 2025-07-27 17:18:27,549] A new study created in memory with name: no-name-da863fff-8642-427d-9b22-99a7f95fda73
[I 2025-07-27 17:18:59,757] Trial 0 finished with value: 0.6204136240175848 and parameters: {'n_estimators': 812, 'max_depth': 29, 'min_samples_split': 15, 'min_samples_leaf': 12, 'max_features': 0.75, 'bootstrap': False}. Best is trial 0 with value: 0.6204136240175848.


🏃 View run Trial - 0 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/cf570a9a4524420b96f5b318e749ee99
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:19:03,111] Trial 1 finished with value: 0.5817238950076484 and parameters: {'n_estimators': 139, 'max_depth': 30, 'min_samples_split': 17, 'min_samples_leaf': 5, 'max_features': 0.75, 'bootstrap': True}. Best is trial 1 with value: 0.5817238950076484.


🏃 View run Trial - 1 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/11a13852945e4ade970a50b0ac2f8e8d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:19:13,808] Trial 2 finished with value: 0.6263932402455473 and parameters: {'n_estimators': 1263, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_features': 'log2', 'bootstrap': True}. Best is trial 1 with value: 0.5817238950076484.


...

🏃 View run Trial - 47 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/92b071b668d84a59be67f4f3bdc0b76a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:43:35,868] Trial 48 finished with value: 0.5556130336390577 and parameters: {'n_estimators': 490, 'max_depth': 29, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': False}. Best is trial 42 with value: 0.4817568655961482.


🏃 View run Trial - 48 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/3a3fd511335a44c7baff1b24df9df124
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:43:46,994] Trial 49 finished with value: 0.5824100386410297 and parameters: {'n_estimators': 727, 'max_depth': 30, 'min_samples_split': 20, 'min_samples_leaf': 7, 'max_features': 'log2', 'bootstrap': False}. Best is trial 42 with value: 0.4817568655961482.


🏃 View run Trial - 49 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/bde96c88977b4042993c55a2955aefd9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
Melhores hiperparâmetros:
{'n_estimators': 1294, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False}
Melhor RMSE: 0.4818
🏃 View run Random Forest optimization at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/c34602bdae614615b6645e8fddd101ef
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
